# Best model coefficients by mortality group
- Standardised coefficient plots count/cmass
- Coefficient table for count/cmass

In [6]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [11]:
# Load libraries
import numpy as np
import pandas as pd
import sys
import os
import pickle
import glob

from classes import boundaries as bd

%config Completer.use_jedi = False 


#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'


#--- Arguments
if run == 'command line':
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    date_exp_str, alpha, standardise_covars = arguments.split(',')
    
    alpha              = float(alpha)   
    standardise_covars = bool(standardise_covars)

elif run == 'manual':
    standardise_covars = False        # Standardise the covariates before running model selection
    date_exp_str = '230207_Management'
    alpha = 0.01


In [12]:
#---------------#
#  Import data  #
#---------------#
date_exp_str_original = date_exp_str

levels             = ['Site', 'Species']
path_levels        = ['site_reg_results/','spe_reg_results/']
mtf_type           = ['count', 'cmass']
climate_data_plot  = {'CRUclim':'$CRU_{clim}$',
                      'CRUNCEPv7':'$CRU_{obs}$',
                      'CHELSA30s':'$CHELSA_{30s}$',
                      'WorldClim30s':'$WorldClim_{30s}$',
                      'WorldClim10m':'$WorldClim_{10m}$',}

mortality_groups = ['M_All','M_Fire','M_NoFire','M_Other']

if standardise_covars:
    sorter       = ['DBH_mean','MAT','Humidity','PFT']
else:
    sorter       = ['Intercept','DBH_mean','MAT','Humidity','PFT', 'Managed']
    
with open(date_exp_str+'_Table4_df.pkl', 'rb') as f:
    table4_df = pickle.load(f)
table4_df = table4_df.reset_index()    

display(table4_df) 

Level Mortality                                  model    r2        RMSE  \
                                                            count cmass count   
0     Site     M_All         MTF ~ MAT + DBH_mean + Managed  0.36  0.42  23.5   
1     Site    M_Fire                         MTF ~ DBH_mean  0.55  0.48  21.0   
2     Site  M_NoFire                              MTF ~ MAT  0.32  0.38  23.0   
3     Site   M_Other                              MTF ~ MAT  0.37  0.43  24.4   
4  Species     M_All  MTF ~ MAT + Humidity + PFT + DBH_mean  0.65  0.63  25.7   
5  Species    M_Fire                         MTF ~ DBH_mean  0.71  0.66  38.3   
6  Species  M_NoFire             MTF ~ MAT + PFT + DBH_mean  0.54  0.54  12.4   
7  Species   M_Other             MTF ~ MAT + PFT + DBH_mean  0.58  0.59  12.6   

           ID        N_obs  
  cmass count cmass         
0  49.3     1     0   64.0  
1  54.9     0     0   17.0  
2  45.0    28    26   47.0  
3  47.8    33    34   37.0  
4  43.7     1     1  107.0  
5  67.8     1     0   37.0  
6  18.9     3     3   70.0  
7  19.6     2     0   61.0

In [13]:
# --- Specifify experiments for import
spe_all_data_f = [date_exp_str+'',               'CRUNCEPv7', alpha]
spe_fire_f     = [date_exp_str+'_FireOnly',      'CRUNCEPv7', alpha]
spe_no_fire_f  = [date_exp_str+'_NoFire',        'CRUNCEPv7', alpha]
spe_other      = [date_exp_str+'_NoInsectsFire', 'CRUNCEPv7', alpha]

species_import = [spe_all_data_f,spe_fire_f,spe_no_fire_f,spe_other] 
sort_dict      = {i:j for i,j  in zip(sorter,np.arange(len(sorter)))}


#--- Initialise
data_mtf_type = np.empty((len(levels),len(mtf_type),),dtype='object')
data          = np.empty((len(levels),len(mortality_groups)),dtype='object')



if standardise_covars:
    st=1
    levels = levels[1:]
else:
    st=0
    levels = levels

for l,lev in enumerate(levels): 
    print(lev)
    if standardise_covars:
        l += 1

    
    table4_df_ss = table4_df[table4_df.Level == lev].copy()
    
    arr_coeff = np.empty((len(mortality_groups),len(mtf_type)),dtype='object')
    
    for i,keys in enumerate(species_import):
        
        # import data
        date_exp_str, climate_data, alpha = keys
        import_str = path_levels[l]+date_exp_str+'_S*_'+str(alpha)+'_'+climate_data+'.pkl'
        import_str = glob.glob(import_str)[0]
        
        with open(import_str, 'rb') as f:
            data[l,i] = pickle.load(f)
        
        for t,tpe in enumerate(mtf_type):
            if (st == 1): # Standardised covars
                continue
            else:        # Actual model coefficients
                modN = table4_df_ss.loc[(table4_df_ss.Mortality == mortality_groups[i]), 
                                        [('ID',tpe)]].values[0][0]
            df = pd.DataFrame()
            df['Mortality']    = np.nan
            
            if np.isnan(modN): # Management subset: Fire Mortality at site level
                df['Value']        = np.nan
                df['Std_error']    = np.nan
            else:
                df['Value']        = data[l,i][st,t,1][modN].params
                df['Std_error']    = data[l,i][st,t,1][modN].bse
            
            df.loc[df['Mortality'].isnull(), 'Mortality'] = mortality_groups[i]
            df['MTF_type']     = tpe
            df['Level']        = lev
            arr_coeff[i,t] = df

    for t in range(2):
        data_mtf_type[l,t]         = pd.concat(arr_coeff[:,t],axis=0)
        data_mtf_type[l,t]         = data_mtf_type[l,t].reset_index()
        data_mtf_type[l,t].columns = ['Coefficients','Mortality','Value','Std_error','MTF_type','Level' ]

Site
Species


In [14]:
# Get unstandardised coefficients in a table
if not standardise_covars:
    df = pd.DataFrame()
    for l,lev in enumerate(levels):
        
        # Initialise    
        df_coef_paper = pd.DataFrame()

        for t,data in enumerate(data_mtf_type[l]):
            ss = data[['Mortality', 'Coefficients', 'Value', 'Std_error','MTF_type']]
            ss = ss.round(2)
            ss['MTF_type'] = mtf_type[t]
            ss['Val_std'] =[str(i)+' ('+str(j)+')' for i,j in zip(ss.Value.to_list(),ss.Std_error.to_list())]
            ss_idx = ss.set_index(['Mortality', 'Coefficients','MTF_type'])
            ss_unstacked = ss_idx['Val_std'].unstack(level=[1,2])

            df_coef_paper = pd.concat([df_coef_paper,ss_unstacked],axis=1)

        df_coef_paper = df_coef_paper.sort_index(axis=1, level=[0, 1],ascending=False)
        df_coef_paper = df_coef_paper.replace('nan (nan)',np.nan)
        df_coef_paper.reset_index(inplace=True)
        df_coef_paper['Level'] = lev
        df = pd.concat([df,df_coef_paper])
    df.set_index(['Level', 'Mortality'],inplace=True)
    df = df[['Intercept','DBH_mean','MAT','PFT','Humidity','Managed']]
    display(df)
    # Store for management influence comparison
    if ('Full' in date_exp_str_original):
        with open(date_exp_str_original+'_Table5_Full_covariate_coefficients.pkl','wb') as f:
            pickle.dump(df,f)
    elif ('Management' in date_exp_str_original):
        with open(date_exp_str_original+'_Table5_Management_covariate_coefficients.pkl','wb') as f:
            pickle.dump(df,f)

Coefficients         Intercept                  DBH_mean               \
MTF_type                 count        cmass        count        cmass   
Level   Mortality                                                       
Site    M_All      2.64 (0.22)  3.25 (0.23)  0.03 (0.01)  0.03 (0.01)   
        M_Fire     1.44 (0.37)   2.12 (0.4)  0.05 (0.01)  0.05 (0.01)   
        M_NoFire   3.25 (0.15)   3.9 (0.16)          NaN          NaN   
        M_Other     3.4 (0.16)  4.06 (0.17)          NaN          NaN   
Species M_All       1.4 (0.22)  1.61 (0.24)   0.04 (0.0)   0.04 (0.0)   
        M_Fire     1.49 (0.22)  1.82 (0.25)   0.05 (0.0)  0.05 (0.01)   
        M_NoFire   2.51 (0.16)  2.82 (0.18)  0.02 (0.01)  0.02 (0.01)   
        M_Other    2.53 (0.17)   2.4 (0.29)  0.02 (0.01)  0.02 (0.01)   

Coefficients                MAT                        PFT               \
MTF_type                  count         cmass        count        cmass   
Level   Mortality                                                         
Site    M_All      -0.09 (0.02)  -0.11 (0.02)          NaN          NaN   
        M_Fire              NaN           NaN          NaN          NaN   
        M_NoFire   -0.11 (0.02)  -0.13 (0.02)          NaN          NaN   
        M_Other    -0.12 (0.03)  -0.15 (0.03)          NaN          NaN   
Species M_All      -0.13 (0.02)  -0.14 (0.02)  0.46 (0.14)   0.5 (0.16)   
        M_Fire              NaN           NaN          NaN          NaN   
        M_NoFire   -0.13 (0.02)  -0.14 (0.02)  0.44 (0.14)  0.44 (0.15)   
        M_Other    -0.14 (0.02)  -0.16 (0.02)  0.52 (0.14)  0.53 (0.15)   

Coefficients          Humidity                    Managed                
MTF_type                 count        cmass         count         cmass  
Level   Mortality                                                        
Site    M_All              NaN          NaN  -0.69 (0.21)  -0.78 (0.22)  
        M_Fire             NaN          NaN           NaN           NaN  
        M_NoFire           NaN          NaN           NaN           NaN  
        M_Other            NaN          NaN           NaN           NaN  
Species M_All      0.32 (0.08)  0.36 (0.09)           NaN           NaN  
        M_Fire             NaN          NaN           NaN           NaN  
        M_NoFire           NaN          NaN           NaN           NaN  
        M_Other            NaN   0.21 (0.1)           NaN           NaN